# Explore P2P Data
Setup

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# visualization
import streamlit as st
import networkx as nx
import plotly

# warning suppresion
import warnings; warnings.simplefilter('ignore')


# We can also use Snowpark for our analyses!
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

In [ ]:
# user node table
user_df = session.table(f'{session.get_current_database()}.{session.get_current_schema()}.p2p_users').to_pandas()
transaction_pagerank_df = (session.table(f'{session.get_current_database()}.{session.get_current_schema()}.p2p_transaction_pagerank')
                           .to_pandas().rename(columns={'SCORE':'TRANSACTION_PAGERANK'}))
louvain_df = (session.table(f'{session.get_current_database()}.{session.get_current_schema()}.p2p_users_vw_lou').to_pandas()
              .rename(columns={'COMMUNITY_ID':'LOUVAIN_COMM_ID'}))
user_df = user_df.merge(transaction_pagerank_df, on="NODEID").merge(louvain_df, on="NODEID")
user_df

In [ ]:
transactions_df = session.table(f'{session.get_current_database()}.{session.get_current_schema()}.p2p_agg_transactions').to_pandas()
louvain_count_df = louvain_df.groupby('COMMUNITY').count().reset_index().rename(columns={'NODEID': 'USER_COUNT'}).sort_values('USER_COUNT', ascending=False)
louvain_count_df['ST_SELECTOR_LABEL'] = louvain_count_df.apply( lambda row: f'id: {row.COMMUNITY}, user_count: {row.USER_COUNT}',axis=1)
louvain_count_df

In [ ]:
import networkx as nx
import plotly.graph_objects as go


def make_graph_from_louvain_ids(louvain_ids):
    n_df = user_df[user_df.LOUVAIN_COMM_ID.isin(louvain_ids)]
    user_ids = n_df.NODEID.tolist()
    e_df = transactions_df[(transactions_df.SOURCENODEID.isin(user_ids)) & (transactions_df.TARGETNODEID.isin(user_ids))]
    
    G = nx.from_pandas_edgelist(e_df, source='SOURCENODEID', target='TARGETNODEID')
    nx.set_node_attributes(G, dict(zip(n_df.USER_ID, n_df.to_dict(orient="records"))))
    return G

def size_scale(lst, bounds=(5,10)):
    mx = max(lst)
    mn = min(lst)
    d = mx-mn
    return [(bounds[1] - bounds[0])*((i - mn)/(d)) + bounds[0] for i in lst]
    
def make_graph_from_louvain_ids(louvain_ids, scale_prop):
    n_df = user_df[user_df.COMMUNITY.isin(louvain_ids)]
    user_ids = n_df.NODEID.tolist()
    e_df = transactions_df[(transactions_df.SOURCENODEID.isin(user_ids)) & (transactions_df.TARGETNODEID.isin(user_ids))]
    
    G = nx.from_pandas_edgelist(e_df, source='SOURCENODEID', target='TARGETNODEID', create_using=nx.DiGraph())
    nx.set_node_attributes(G, dict(zip(n_df.NODEID, n_df[['NODEID', scale_prop]].to_dict(orient="records"))))
    return G


def plot_graph(G, title="subgraph", scale_prop="CARD_COUNT"):
    pos = nx.spring_layout(G)
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
    
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')
    
    node_x = []
    node_y = []
    node_influence = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_influence.append(round(G.nodes[node][scale_prop],3))
    node_size = size_scale(node_influence, (10, 30))

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='Reds',
            reversescale=False,
            color=[],
            size= node_size,
            colorbar=dict(
                thickness=15,
                title=scale_prop,
                xanchor='left'
                #titleside='right'
            ),
            line=dict(width=2,color='DarkSlateGrey')))
    
    node_text = []
    for node in G.nodes():
        node_text.append(str(G.nodes[node]))
    
    node_trace.marker.color = node_influence
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title=title,
                    #titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig


def plot_pr_graph_from_louvain_ids(louvain_ids):
    G = make_graph_from_louvain_ids(louvain_ids, "TRANSACTION_PAGERANK")
    return  plot_graph(G, title=f"Louvain Community: {louvain_ids}", scale_prop="TRANSACTION_PAGERANK")

#Plot Louvain Community using NetworkX - Coming Soon our Neo4j Visualization Python Library

In [ ]:
import streamlit as st
# id: 28656, user_count: 29

st.subheader("Explore Graph Communities")
selected_comm_id = st.selectbox(
   "Select a Community",
   louvain_count_df['ST_SELECTOR_LABEL'].tolist(),
   placeholder="select community id...",
)

if selected_comm_id:
    comm_ids = louvain_count_df[louvain_count_df['ST_SELECTOR_LABEL'] == selected_comm_id]['COMMUNITY'].tolist()
    st.plotly_chart(plot_pr_graph_from_louvain_ids(comm_ids))
    

In [ ]:
SELECT * from P2P_DEMO.PUBLIC.P2P_AGG_TRANSACTIONS;